# Getting articles from wordpress
1. Request the articles from wordpress
2. Can only request 100 at a time so we need to do a loop
3. Clean up html, weird characters so it does not impact our results and stuff that comes with instagrams embeed
4. Export it to a csv that later is cleaned up 

In [6]:
import requests
def get_data(page, per_page, filtering = False):
    
    url =  f"https://www.plateapr.com/wp-json/wp/v2/posts?page={page}&per_page={per_page}"
    if filtering:
        url =  url+"&_fields[]=title&_fields[]=content&_fields[]=link&_fields[]=date&_fields[]=author&_fields[]=type"

    print(f"Requesting... {url}")
    result = requests.get(url)
    if result.ok:
        print(f"Sucess request... {url}")
    else:
        print(url)
        raise Exception(
            f"Request failed: {result.text}")
    return result

In [7]:
per_page = 100
page = 1

there_are_more_pages = True
articles = []
while(there_are_more_pages):
    try:
        result = get_data(page, per_page, True)
        for article in result.json():
            articles.append(article)
        page = page + 1
    except:
        there_are_more_pages = False
        break


Requesting... https://www.plateapr.com/wp-json/wp/v2/posts?page=1&per_page=100&_fields[]=title&_fields[]=content&_fields[]=link&_fields[]=date&_fields[]=author&_fields[]=type
Sucess request... https://www.plateapr.com/wp-json/wp/v2/posts?page=1&per_page=100&_fields[]=title&_fields[]=content&_fields[]=link&_fields[]=date&_fields[]=author&_fields[]=type
Requesting... https://www.plateapr.com/wp-json/wp/v2/posts?page=2&per_page=100&_fields[]=title&_fields[]=content&_fields[]=link&_fields[]=date&_fields[]=author&_fields[]=type
Sucess request... https://www.plateapr.com/wp-json/wp/v2/posts?page=2&per_page=100&_fields[]=title&_fields[]=content&_fields[]=link&_fields[]=date&_fields[]=author&_fields[]=type
Requesting... https://www.plateapr.com/wp-json/wp/v2/posts?page=3&per_page=100&_fields[]=title&_fields[]=content&_fields[]=link&_fields[]=date&_fields[]=author&_fields[]=type
https://www.plateapr.com/wp-json/wp/v2/posts?page=3&per_page=100&_fields[]=title&_fields[]=content&_fields[]=link&_fi

In [49]:
import re
CLEANR = re.compile('<.*?>') 

def remove_html(raw_html):
    cleantext = re.sub(CLEANR, '', raw_html)
    return cleantext


def remove_characters(raw_string):
    
    cleantext = re.sub('&#[0-9][0-9][0-9][0-9];', '', raw_string)
    cleantext = re.sub('&#[0-9][0-9][0-9];', '', cleantext)
    cleantext = re.sub('&#[0-9][0-9];', '', cleantext)
    cleantext = re.sub('&nbsp;', '', cleantext)
    cleantext = re.sub('&amp;', '&', cleantext)
    cleantext = re.sub('// ', '', cleantext)
    cleantext = cleantext.replace("|", "")
    cleantext = cleantext.replace("&#8217;", "")
    cleantext = cleantext.replace("“", "")
    cleantext = cleantext.replace("”", "")
    cleantext = cleantext.replace("’", "")
    cleantext = cleantext.replace("#", "")
    cleantext = cleantext.replace("-", " ")
    cleantext = cleantext.replace("ü", "u")

    
    return cleantext

def remove_spacing(raw_string):
    raw_string = raw_string.replace("\n", " ")
    return re.sub("\s\s+", " ", raw_string)


def remove_emojis(data):

    emoj = re.compile("["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
    u"\U00002500-\U00002BEF"  # chinese char
    u"\U00002702-\U000027B0"
    u"\U00002702-\U000027B0"
    u"\U000024C2-\U0001F251"
    u"\U0001f926-\U0001f937"
    u"\U00010000-\U0010ffff"
    u"\u2640-\u2642" 
    u"\u2600-\u2B55"
    u"\u200d"
    u"\u23cf"
    u"\u23e9"
    u"\u231a"
    u"\ufe0f"  # dingbats
    u"\u3030"
                  "]+", re.UNICODE)
    return re.sub(emoj, '', data)


def remove_dollars(data):
    data = data.replace("$ ", "")
    data = data.replace("$", "")

    return  data

def remove_at_tags(data):
    cleantext=  re.sub(r'\s*\(@.+?\)\s*', ' ', data)
    cleantext = cleantext.replace("(", "")
    cleantext = cleantext.replace(")", "")
    return cleantext

def remove_instagram_text(data):
    return data.replace("View this post on Instagram A post shared by ", "")


to_replace = {
                "playas":"playa",
                "cocteles": "alcohol", 
                "tragos":'alcohol', 
                'cerveza':"alcohol", 
                'cervezas':"alcohol",
                "ron":"alcohol",
                "margaritas": "alcohol",
                "mojitos":'alcohol',
                "restaurantes":"comida",
                "restaurante":"comida",
                
             }

def append_words(data):
    for key in to_replace:
        number = data.count(key)
        data = data + (f'{to_replace[key]} ' * number) # Appending number of times the synonyms
    return data
    

def replace_words(data):
    for key in to_replace:
        data = data.replace(key, to_replace[key])
    return data
    

def clean_article(article):
    content = article['title']['rendered'] + ' ' + article['content']['rendered']
    content =  article['content']['rendered']

    art = dict()
    art['title'] = article['title']['rendered']
    art['content'] =  remove_instagram_text(remove_at_tags(remove_dollars(remove_emojis(remove_spacing(remove_characters(remove_html(content))))).strip()))
    art['link'] = article['link']
    art['date'] = article['date']
    art['author'] = article['author']
    art['type'] = article['type']

    return art
    

In [50]:
cleaned = [clean_article(a) for a in articles]
cleaned[10]

{'title': 'Nacho Libre presenta nuevo menú de margaritas y tacos',
 'content': 'Nacho Libre Cantina Mexicana, el restaurante conocido por su refrescante coctelería con tequila renovó su menú completamente, con nuevas opciones de margaritas y tacos suaves. Entre los sabores de la nueva oferta de tacos se destacan los tacos de brisket, con crema de aguacate, cebolla, cilantro y potato shoestrings. Además, el taco de asada, un corte de churrasco en salsa verde acompañado por cebolla y cilantro. Sin embargo, la estrella del nuevo menú es la plantilla de costra. Es decir, una capa crocante de queso derretido a la plancha que va encima de la plantilla del taco. Nuevo menú de margaritas en Nacho Libre Para darle continuidad a su exitoso menú de margaritas, Nacho Libre añadió nuevas fusiones de coctelería que van desde sabores de frutas tradicionales hasta tonalidades más innovadoras como café y chocolate. Nacho Libre Cantina Mexicana Para los amantes del espresso martini, Nacho Libre les ofre

In [51]:
import pandas as pd
data = pd.DataFrame.from_records(cleaned)
data.to_csv('./data/wp_posts2.csv', index=False)

# Further Clean Up

In [52]:
to_clean = pd.read_csv('./data/text-similarity-documents.csv')

In [56]:
to_clean['content']= [remove_instagram_text(a) for a in to_clean['content']]

In [57]:
import pandas as pd
data = pd.DataFrame.from_records(to_clean)
data.to_csv('./data/wp_posts_cleaned.csv', index=False)

In [58]:
len(data)

184